[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/gaussian-splatting-colab/blob/main/gaussian_splatting_viewer_colab.ipynb)

In [2]:
#!git clone https://github.com/camenduru/splat
# Check if the operating system is macOS
import platform
import subprocess

if platform.system() == 'Darwin':
    # Install cloudflared using Homebrew on macOS
    subprocess.run(['brew', 'install', 'cloudflared'])
else:
    # Original commands for Linux
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O /content/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
beautysh
bibtex-tidy
cbfmt
cljfmt
dotnet@8
energy
mergiraf
neocmakelsp
screenpipe
sigsum-go
==> New Casks
beyond-compare@4
djstudio
font-departure-mono-nerd-font
font-maple-mono-nf
font-montserrat-underline
kimis
last-window-quits
pixpin
teleport

You have 3 outdated formulae and 1 outdated cask installed.



==> Downloading https://ghcr.io/v2/homebrew/core/cloudflared/manifests/2024.11.0
==> Fetching cloudflared
==> Downloading https://ghcr.io/v2/homebrew/core/cloudflared/blobs/sha256:0ecd402f4ed52c392f3031a5881b5a974ab9084fc098bba46d305bdb561cf858
==> Pouring cloudflared--2024.11.0.arm64_sequoia.bottle.tar.gz
🍺  /opt/homebrew/Cellar/cloudflared/2024.11.0: 8 files, 36.1MB
==> Running `brew cleanup cloudflared`...
Disable this behaviour by setting HOMEBREW_NO_INSTALL_CLEANUP.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).


In [3]:
import atexit, requests, time, re, os
from random import randint
from threading import Timer
from queue import Queue

In [5]:
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
#%cd /content/splat
!python -m http.server 7860

<>:8: SyntaxWarning: invalid escape sequence '\/'
<>:8: SyntaxWarning: invalid escape sequence '\/'
/var/folders/zd/6gytrrmd0jx86dlhrkrlp1d80000gn/T/ipykernel_2890/1956468613.py:8: SyntaxWarning: invalid escape sequence '\/'
  tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")


https://salem-saddam-jail-flash.trycloudflare.com
Serving HTTP on :: port 7860 (http://[::]:7860/) ...
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:53] "GET / HTTP/1.1" 200 -
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:53] code 404, message File not found
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:53] "GET /favicon.ico HTTP/1.1" 404 -
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:58] "GET / HTTP/1.1" 200 -
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:58] "GET / HTTP/1.1" 200 -
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:58] "GET / HTTP/1.1" 200 -
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:59] "GET / HTTP/1.1" 200 -
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:59] code 404, message File not found
::ffff:127.0.0.1 - - [19/Nov/2024 15:56:59] "GET /favicon.ico HTTP/1.1" 404 -
::ffff:127.0.0.1 - - [19/Nov/2024 15:57:02] "GET /README.md HTTP/1.1" 200 -
::ffff:127.0.0.1 - - [19/Nov/2024 15:57:18] "GET /gaussian_splatting_colab.ipynb HTTP/1.1" 200 -
^C

Keyboard interrupt received, exiting.
